In [ ]:
model_path = ""
dataset_path = ""
insdata_path = ""
best_params_path = ""
cross_val_stats_path = ""
experiment_description_path = ""

In [ ]:
# import os
# os.chdir("/Users/peterzvirinsky/src/git/GraphSlicesExperiments/dynamic_pagerank/")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from common import load_markup
import seaborn
import keras
from plots import plot_ice_grid
from pycebox.ice import ice

seaborn.set_style("whitegrid")

# Experiment

In [ ]:
experiment_description = load_markup(experiment_description_path)
dataset_df = pd.read_csv(dataset_path)
model = model = keras.models.load_model(model_path)
best_params = json.load(open(best_params_path))
cv_stats_df = pd.read_csv(cross_val_stats_path)

In [ ]:
display(experiment_description)

In [ ]:
model.summary()

### Basic eval

In [ ]:
print("{}-fold cross val results:".format(cv_stats_df.shape[0]))
cv_stats_df.mean().to_frame(name="Score")

### Test predictions

In [ ]:
X = dataset_df.drop("year_0", axis=1)
y = dataset_df["year_0"]

In [ ]:
predictions = model.predict(X)
dataset_df["prediction"] = predictions
dataset_df["prediction_abs_err"] = (dataset_df["prediction"] - dataset_df["year_0"]).abs()

In [ ]:
dataset_df.sort_values(by="prediction_abs_err", ascending=False)

## Prediction vs true plots

In [ ]:
insdata_df = pd.read_csv(insdata_path)
assert insdata_df.shape[0] == dataset_df.shape[0]

In [ ]:
def plot_true_vs_pred_for(node_string_id):
    dataset_df[insdata_df.stringId == node_string_id]["year_0"].reset_index(drop=True).plot(figsize=(12, 4), title="True")
    plt.show()
    dataset_df[insdata_df.stringId == node_string_id]["prediction"].reset_index(drop=True).plot(figsize=(12, 4), title="Prediction")
    plt.show()

### Ge Money Bank

In [ ]:
plot_true_vs_pred_for("gemoneybank")

### T mobile

In [ ]:
plot_true_vs_pred_for("t-mobileczechrepublic")

### Cetelem

In [ ]:
plot_true_vs_pred_for("cetelemčr")

### Bohemia Faktoring

In [ ]:
plot_true_vs_pred_for("bohemiafaktoring")

## ICE and PDP plots

In [ ]:
X_unique = X.drop_duplicates().copy()
train_ice_dfs = {feat: ice(data=X_unique, column=feat, predict=model.predict) 
                 for feat in X.columns}

In [ ]:
fig = plot_ice_grid(train_ice_dfs, X_unique, X.columns,
                    ax_ylabel='Pred. AV %ile', alpha=0.3, plot_pdp=True,
                    pdp_kwargs={'c': 'red', 'linewidth': 3},
                    linewidth=0.5, c='dimgray')
fig.tight_layout()
fig.suptitle('ICE plots (training data)')
fig.subplots_adjust(top=0.89);